In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import DepthwiseConv2D, Conv2D, BatchNormalization, ReLU, GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.utils import class_weight
from collections import Counter


In [22]:
data_dir = '/content/drive/MyDrive/UCMerced_LandUse/UCMerced_LandUse/Images_converted'
img_height, img_width = 224, 224
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2)

In [23]:
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

Found 1680 images belonging to 21 classes.
Found 420 images belonging to 21 classes.


In [24]:
counter = Counter(train_generator.classes)
total = sum(counter.values())
class_weights_dict = {cls: total/(len(counter) * count) for cls, count in counter.items()}

In [25]:
def build_googlenet_with_dwsc(input_shape, num_classes):
    base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = DepthwiseConv2D(kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=outputs)
    return model

input_shape = (img_height, img_width, 3)
num_classes = train_generator.num_classes
model = build_googlenet_with_dwsc(input_shape, num_classes)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [26]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

In [27]:
epochs = 5

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs,
    class_weight=class_weights_dict,
    callbacks=[reduce_lr]
)


Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


53/53 ━━━━━━━━━━━━━━━━━━━━ 1616s 27s/step - accuracy: 0.2671 - loss: 2.5784 - val_accuracy: 0.5667 - val_loss: 2.7571 - learning_rate: 0.0010
Epoch 2/5
53/53 ━━━━━━━━━━━━━━━━━━━━ 26s 445ms/step - accuracy: 0.6932 - loss: 1.1467 - val_accuracy: 0.7667 - val_loss: 2.4031 - learning_rate: 0.0010
Epoch 3/5
53/53 ━━━━━━━━━━━━━━━━━━━━ 27s 450ms/step - accuracy: 0.7841 - loss: 0.7674 - val_accuracy: 0.7738 - val_loss: 2.0739 - learning_rate: 0.0010
Epoch 4/5
53/53 ━━━━━━━━━━━━━━━━━━━━ 27s 452ms/step - accuracy: 0.7900 - loss: 0.6836 - val_accuracy: 0.8071 - val_loss: 1.6854 - learning_rate: 0.0010
Epoch 5/5
53/53 ━━━━━━━━━━━━━━━━━━━━ 26s 438ms/step - accuracy: 0.8093 - loss: 0.6032 - val_accuracy: 0.7976 - val_loss: 1.3948 - learning_rate: 0.0010


In [28]:
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')


14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 373ms/step - accuracy: 0.8148 - loss: 1.3165
Validation Loss: 1.3375126123428345
Validation Accuracy: 0.811904788017273
